In [2]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql

In [16]:
def read_config(p:str)->dict:
    with open(p, 'r') as f:
        config=f.readlines()
    config_dict={}

    for l in config:
        idx =l.index('=')
        k=l[:idx] 
        v=l[idx+1:]
        config_dict[k]=v.rstrip()
    return config_dict

In [18]:
db_config=read_config('./db_config')

In [33]:
with conn.cursor() as cursor:
    sql ='DELETE FROM `Students` WHERE `StudentID`=%s'
    cursor.execute(sql, (8,))
conn.commit()

In [49]:
with conn.cursor() as cursor:
    sql = 'SELECT * FROM `Students` LIMIT %s OFFSET %s' 
    cursor.execute(sql, (5,0)) 
    result = cursor.fetchall()

result

((1, '홍길동', 30, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 27, '경기'),
 (5, '김형석', 32, '서울'))

In [69]:
def pagination(page, page_size=5):
    limit = page_size
    offset=page-1

    with conn.cursor() as cursor:
        sql = 'SELECT * FROM `Students` LIMIT %s OFFSET %s' 
        cursor.execute(sql, (limit,offset)) 
        result = cursor.fetchall()

    return result

In [70]:
pagination(1)

((1, '홍길동', 30, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 27, '경기'),
 (5, '김형석', 32, '서울'))

In [71]:
pagination(2)

((2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 27, '경기'),
 (5, '김형석', 32, '서울'),
 (6, 'JASON', 36, '서울'))

In [73]:
with conn.cursor() as cursor:
    sql ='INSERT INTO `student`(`name`, `email`, `phone`, `major`)\
        VALUES (%s, %s, %s, %s)'
    cursor.execute(sql, ("김형석", "hyungsuk0815@gmail.com", "010-9605-1672", "수학"))
conn.commit()

In [87]:
with conn.cursor() as cursor:
    sql ='INSERT INTO `student`(`name`) VALUES (%s)'
    cursor.execute(sql, ("손준현"))
conn.commit()

In [91]:
with conn.cursor() as cursor:
    sql = 'UPDATE `student` SET `major`=%s WHERE `id`=%s' 
    cursor.execute(sql, ("산업공학", 6))
    conn.commit()

In [92]:
with conn.cursor() as cursor:
    sql ='DELETE FROM `student` WHERE `ID`=%s'
    cursor.execute(sql, (5,))
conn.commit()

In [102]:
def insert(table_name, columns, values):
    with conn.cursor() as cursor:
        sql ='INSERT INTO `{}`({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s']*len(values)))
        cursor.execute(sql, values)
    result= conn.commit()
    return result

In [145]:
class MariaDB:
    def __init__(self, host, user, password, database, charset, port=3306):
        self.host=host
        self.port=int(port)
        self.conn=pymysql.connect(
                                host=host,
                                user=user,
                                password=password,
                                database=database,
                                charset=charset,
                                port=port)
        return
        

    def insert(self, table_name, columns, values):
        with self.conn.cursor() as cursor:
            sql ='INSERT INTO `{}`({}) VALUES ({})'.format(table_name,
                                                            ','.join(columns),
                                                              ','.join(['%s']*len(values)))
            cursor.execute(sql, values)
        self.conn.commit()
        return True
    
    def insert_many(self, table_name, columns, values):
        with self.conn.cursor() as cursor:
            sql ='INSERT INTO `{}`({}) VALUES ({})'.format(table_name,
                                                            ','.join(columns),
                                                              ','.join(['%s']*len(values[0])))
            cursor.executemany(sql, values)
        self.conn.commit()
        return True
    
    def select(self, table_name, columns, page=None, page_size=10):
        if page:
            limit = page_size
            offset = page_size*(page-1)
            with self.conn.cursor() as cursor:
                sql = 'SELECT {} FROM {} LIMIT %s OFFSET %s'.format(','.join(columns),
                                                                     table_name) 
                cursor.execute(sql, (limit,offset)) 
                result = cursor.fetchall()
        else:
            with self.conn.cursor() as cursor:
                sql = 'SELECT {} FROM {}'.format(','.join(columns),
                                                table_name) 
                cursor.execute(sql) 
                result = cursor.fetchall()

        return result
    
    def update(self, table_name, set_column, set_value, target_id):
        with self.conn.cursor() as cursor:
            sql = 'UPDATE {} SET {}=%s WHERE id={}'.format(table_name,
                                                            set_column,
                                                              target_id)
            cursor.execute(sql, (set_value, ))
        self.conn.commit()
        return True

In [142]:
db=MariaDB(**db_config)